# Dataset Generation

In [2]:
import random
import pandas as pd
def generate_puzzle_player_stats():
  
  numbers = [1,2,3,4] 

  completion_time = random.randrange(1, 30)
  hints = random.randint(0, 3)
  is_completed = random.randint(0, 1)

  plevel = numbers[random.randint(0, len(numbers) - 1)]
  level = plevel

  # Calculate the level.    
  if completion_time <= 3  and (hints == 0) and is_completed == 1 and plevel==1:
    level = 2
  elif completion_time <= 3  and (hints != 0) and is_completed == 1 and plevel==1:
    level = 1
  elif plevel==3  and is_completed== 0:
    level = 1


  elif (completion_time <= 7 ) and (hints ==0) and is_completed== 1 and plevel==2:
    level = 3
  elif (completion_time <= 7 ) and (hints !=0) and is_completed == 1 and plevel==2:
    level = 2
  elif plevel==7 and is_completed == 0:
    level = 1

  elif  (completion_time <= 15 ) and (hints == 0) and is_completed ==1 and plevel == 3:
    level = 4
  elif   (completion_time <= 15 ) and (hints != 0) and is_completed ==1 and plevel == 3:
    level = 3
  elif  plevel==15 and is_completed == 0:
    level = 2

  elif  (completion_time <= 30 ) and (hints == 0) and is_completed ==1 and plevel == 4:
    level = 4
  elif (completion_time <= 30 ) and (hints != 0) and is_completed ==1 and plevel == 4:
    level = 3
  elif  plevel==30 and is_completed == 0:
    level = 3

  else :
    level = 1

  return {
    "completion_time": completion_time,
    "hints": hints,
    "mistakes": 0,
    "is_completed": is_completed,
    "previous_level":plevel,
    "level": max(level,1)
  }

def generate_puzzle_players_stats_dataset(num_samples):
  """Generates a dataset of Sudoku players stats samples.

  Args:
    num_samples: The number of samples to generate.

  Returns:
    A list of dictionaries, where each dictionary contains the Sudoku player stats
    for a single player.
  """

  puzzle_players_stats_dataset = []
  for i in range(num_samples):
    puzzle_players_stats_dataset.append(generate_puzzle_player_stats())
  return puzzle_players_stats_dataset

# Generate a dataset of 100k Sudoku players stats samples.
puzzle_players_stats_dataset = generate_puzzle_players_stats_dataset(1000000)

df = pd.DataFrame(puzzle_players_stats_dataset)

# Save the DataFrame to a CSV file
df.to_csv("C:\\Users\\faisa\\Documents\\Games_Section\\chess\\CSVs\\puzzle_players_stats.csv", index=False)



# Model Training

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Load your dataset
data = pd.read_csv("C:\\Users\\faisa\\Documents\\Games_Section\\chess\\CSVs\\puzzle_players_stats.csv")


# Split the data into features (X) and the target variable (y)
X = data[["completion_time","hints","mistakes","is_completed","previous_level"]]
y = data["level"]
# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)



# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Fit and transform the labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)
np.save("puzzle_encoded_labels.npy", label_encoder.classes_)


from keras.regularizers import l2

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(5,)),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4,activation='softmax')
])


# Implement learning rate scheduling
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Implement early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# Train the model
history = model.fit(
    X_train, y_train_encoded,
    epochs=100,  # Increase the number of epochs
    batch_size=64,  # Adjust batch size
    validation_data=(X_val, y_val_encoded),
    callbacks=[early_stopping],
    verbose=1  # Use verbose mode 2 for more detailed training output
)

# Save the model to a file
model.save_weights("puzzle_weights.keras")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Test Accuracy: {test_accuracy}")

c:\Users\faisa\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Epoch 1/100
12500/12500 [==============================] - 30s 2ms/step - loss: 0.2314 - accuracy: 0.9634 - val_loss: 0.0703 - val_accuracy: 0.9977
Epoch 2/100
12500/12500 [==============================] - 29s 2ms/step - loss: 0.1171 - accuracy: 0.9792 - val_loss: 0.1353 - val_accuracy: 0.9576
Epoch 3/100
12500/12500 [==============================] - 29s 2ms/step - loss: 0.1035 - accuracy: 0.9822 - val_loss: 0.0530 - val_accuracy: 0.9986
Epoch 4/100
12500/12500 [==============================] - 28s 2ms/step - loss: 0.0941 - accuracy: 0.9835 - val_loss: 0.0486 - val_accuracy: 0.9985
Epoch 5/100
12500/12500 [==============================] - 28s 2ms/step - loss: 0.0841 - accuracy: 0.9855 - val_loss: 0.0400 - val_accuracy: 1.0000
Epoch 6/100
12500/12500 [==============================] - 29s 2ms/step - loss: 0.0767 - accuracy: 0.9868 - val_loss: 0.0375 - val_accuracy: 0.9994
Epoch 7/100
12500/12500 [==============================] - 31s 2ms/step - loss: 0.0701 - accuracy: 0.9879 - val_

# Model Usage

In [8]:
import tensorflow as tf
from keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
import numpy as np

weights_path = {
    "sudoku":"sudoku_weights.keras",
    "missing_words":"missing_words_weights.keras",
    "puzzle":"puzzle_weights.keras" }
classes_path = {
    "sudoku":"sudoku_encoded_labels.npy",
    "missing_words":"missing_words_encoded_labels.npy",
    "puzzle":"puzzle_encoded_labels.npy" }


def predict(features,game):
    """
    features[0] = completion_time 
    features[1] = hints 
    features[2] = mistakes 
    features[3] = is_completed 
    features[4] = previous_level 
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(5,)),
        tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(4,activation='softmax')
    ])
    model.load_weights(weights_path[game])
    features = np.array([features])
    predicted_probs = model.predict(features)
    predicted_class = np.argmax(predicted_probs)
    class_labels = np.load(classes_path[game])
    predicted_level = class_labels[predicted_class]
    return predicted_level


print(predict([10,0,0,1,15],'puzzle'))


1/1 [==============================] - 0s 84ms/step
4
